# tsTools Online: MEaSUREs Training Data Collection

In [98]:
from ipywidgets import Button, HBox, VBox, Label, BoundedIntText, HTML, Dropdown, Layout, FloatText
import bqplot
import datetime
import dateutil.parser
import ee
import ipywidgets
import ipywidgets as widgets
import IPython.display
import numpy as np
import pprint
import pandas as pd
import traitlets
import ipyleaflet
import geopandas
import bqplot.pyplot as plt
from dateutil import parser
import qgrid
import colorsys


from bqplot.interacts import BrushSelector, BrushIntervalSelector

import sqlite3
import os

from bqplot.interacts import BrushSelector, BrushIntervalSelector

# Google sheets API
import gspread
from oauth2client.service_account import ServiceAccountCredentials

import ccd

# Turn off pandas warning
pd.options.mode.chained_assignment = None

# Configure the pretty printing output.
pp = pprint.PrettyPrinter(depth=4)

In [99]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [100]:
# Authenticate to the Earth Engine servers, and initialize the ee module.
ee.Initialize()

In [101]:
# Functions to load collections, merge, filter and calculate statistics

# Filter collection by point and date
def collection_filtering(point, collection_name, year_range, doy_range):
    collection = ee.ImageCollection(collection_name)\
    .filterBounds(point)\
    .filter(ee.Filter.calendarRange(year_range[0], year_range[1], 'year'))\
    .filter(ee.Filter.dayOfYear(doy_range[0],doy_range[1]))
    return collection

# Landsat stack renamers, C1
def stack_renamer_l4_7_C1(img):
    band_list = ['B1', 'B2','B3','B4','B5','B7', 'B6','pixel_qa']
    name_list = ['BLUE','GREEN','RED','NIR','SWIR1','SWIR2','THERMAL','pixel_qa']
    return ee.Image(img).select(band_list).rename(name_list)

def stack_renamer_l8_C1(img):
    band_list = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10','pixel_qa'];
    name_list = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2','THERMAL','pixel_qa'];
    return ee.Image(img).select(band_list).rename(name_list);


# Cloud masking for C1, L4-L7. Operators capitalized to
# avoid confusing with internal Python operators
def cloud_mask_l4_7_C1(img):
    pqa = ee.Image(img).select(['pixel_qa'])
    mask = (pqa.eq(66)).Or(pqa.eq(130))\
    .Or(pqa.eq(68)).Or(pqa.eq(132))
    return ee.Image(img).updateMask(mask)
  
# Cloud masking for C1, L8
def cloud_mask_l8_C1(img):
    pqa = ee.Image(img).select(['pixel_qa'])
    mask = (pqa.eq(322)).Or(pqa.eq(386)).Or(pqa.eq(324))\
    .Or(pqa.eq(388)).Or(pqa.eq(836)).Or(pqa.eq(900))
    return ee.Image(img).updateMask(mask)


# Calculate period statistics, seems to work but it's so ugly!
def calc_period_stats(yr_list, collection, reducer, stat_name, bands):
    def auxfnc(year):
        start = ee.Date(year)
        end = start.advance(step, 'year').advance(-1, 'day')

        return collection\
        .select(bands)\
        .filterDate(start, end)\
        .reduce(reducer)\
        .set({'system:time_start': start, 'system:time_end': end, 'statistic': stat_name})
    period_stats = yr_list.map(auxfnc)
    return period_stats

def fc2df(fc):
    # Convert a FeatureCollection into a pandas DataFrame
    # Features is a list of dict with the output
    features = fc.getInfo()['features']
    dictarr = []
    for f in features:
        # Store all attributes in a dict
        attr = f['properties']
        dictarr.append(attr)

    return pd.DataFrame(dictarr)

def fc2dfgeo(fc):
    # Convert a FeatureCollection into a pandas DataFrame
    # Features is a list of dict with the output
    features = fc.getInfo()['features']

    dictarr = []

    for f in features:
        # Store all attributes in a dict
        attr = f['properties']
        # and treat geometry separately
        attr['geometry'] = f['geometry']
        dictarr.append(attr)

    df = geopandas.GeoDataFrame(dictarr)
    return df

def get_df(collection, coords, band):
    point = ee.Geometry.Point(coords)
    # Sample for a time series of values at the point.
    geom_values = collection.filterBounds(point).select(band).getRegion(geometry=point, scale=30)
    geom_values_list = ee.List(geom_values).getInfo()
    # Convert to a Pandas DataFrame.
    header = geom_values_list[0]
    data = pd.DataFrame(geom_values_list[1:], columns=header)
    data['datetime'] = pd.to_datetime(data['time'], unit='ms', utc=True)
    data.set_index('time')
    data = data.sort_values('datetime')
    data = data[['id', 'datetime', band]]
    return data

def get_df_full(collection, coords):
    point = ee.Geometry.Point(coords)
    # Sample for a time series of values at the point.
    geom_values = collection.filterBounds(point).getRegion(geometry=point, scale=30)
    geom_values_list = ee.List(geom_values).getInfo()
    # Convert to a Pandas DataFrame.
    header = geom_values_list[0]
    data = pd.DataFrame(geom_values_list[1:], columns=header)
    data['datetime'] = pd.to_datetime(data['time'], unit='ms', utc=True)
    data['doy'] = pd.DatetimeIndex(data['datetime']).dayofyear
    color_list = get_color_list()
    data['color'] = [color_list[i] for i in data['doy']]
    data.set_index('time')
    data = data.sort_values('datetime')
    data['ord_time'] = data['datetime'].apply(datetime.date.toordinal)
    data = data[['id', 'datetime', 'ord_time', 'BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'THERMAL', 'pixel_qa', 'doy', 'color']]
    return data


def GetTileLayerUrl(ee_image_object):
    map_id = ee.Image(ee_image_object).getMapId()
    tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    return tile_url_template.format(**map_id)

# Load, filter and merge collections
def get_full_collection(coords, year_range, doy_range):
    point = ee.Geometry.Point(coords)
    l8_renamed = collection_filtering(point, 'LANDSAT/LC08/C01/T1_SR', year_range, doy_range)\
        .map(stack_renamer_l8_C1)
    l8_filtered1 = l8_renamed.map(cloud_mask_l8_C1)

    l7_renamed = collection_filtering(point, 'LANDSAT/LE07/C01/T1_SR', year_range, doy_range)\
        .map(stack_renamer_l4_7_C1);
    l7_filtered1 = l7_renamed.map(cloud_mask_l4_7_C1)

    l5_renamed = collection_filtering(point, 'LANDSAT/LT05/C01/T1_SR', year_range, doy_range)\
        .map(stack_renamer_l4_7_C1)
    l5_filtered1 = l5_renamed.map(cloud_mask_l4_7_C1)


    all_scenes = ee.ImageCollection((l8_filtered1.merge(l7_filtered1)).merge(l5_filtered1)).sort('system:time_start')
    return all_scenes

def get_color_list():
    colors = [['#000000'] * 79 \
             + ['#41b6c4'] * 93 \
             + ['#41ab5d'] * 94 \
             + ['#e31a1c'] * 89 \
             + ['#000000'] * 11][0]

    return colors

In [102]:
# PyCCD Functions
# Functions for running pyccd
def make_df_pyccd(collection, point):
    band_list = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2','THERMAL', 'pixel_qa']
    rename_list = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2','THERMAL', 'pixel_qa']
    info = collection.getRegion(point, 30).getInfo()
    header = info[0]
    data = np.array(info[1:])
    iTime = header.index('time')
    time = [datetime.datetime.fromtimestamp(i/1000) for i in (data[0:,iTime].astype(int))]
    time_new = [t.toordinal() for t in (time)]
    iBands = [header.index(b) for b in band_list]
    yData = data[0:,iBands].astype(np.float)
    df = pd.DataFrame(data=yData, index=list(range(len(yData[:,0]))), columns=rename_list)
    df['time'] = time_new
    return df

def update_df(df, df2):
    df = df.append(df2)
    return df


In [103]:
class interface(object):
    """A class to hold interface for defining sample attributes"""
    
    # Set up database
    dbPath = os.getcwd() + '/sample_database_3'
    if not os.path.isfile(dbPath):
        conn = sqlite3.connect(dbPath)
        c = conn.cursor()
        c.execute('''CREATE TABLE sample3
                 (id real, lat real, lon real, year1 real, year2 real, condition text, change text, class text, water text, bare text, albedo text, use text,
                 height text, transport text, impervious text, density text, vegType1 text, herbaceous text, shrub text, forestPhenology text,
                 leafType text, location text, confidence real, notes text)''')
        print('Database created')
        
    
    # Create widgets
    # Years slider
    years = widgets.IntRangeSlider(value=[2012, 2015], min=1990, max=2018,
            step=1, description='Years:', disabled=False, continuous_update=False,
            orientation='horizontal', readout=True, readout_format='d')

    # Dropdown hierachy
    drop1 = widgets.Dropdown(options=['Persistant Ice?','Yes', 'No'],
                value='Persistant Ice?', description='Decision 3', disabled=False)

    drop2 = widgets.Dropdown(options=['Decision 2'],
                value='Decision 2', description='Decision 4', disabled=False)
     
    drop3 = widgets.Dropdown(options=['Decision 3'],
                value='Decision 3', description='Decision 5', disabled=False)

    drop4 = widgets.Dropdown(options=['Decision 4'],
                value='Decision 4', description='Decision 6', disabled=False)
 
    drop5 = widgets.Dropdown(options=['Decision 5'],
                value='Decision 5', description='Decision 7', disabled=False)

    drop6 = widgets.Dropdown(options=['Decision 6'],
                value='Decision 6', description='Decision 8', disabled=False)
    
    drop7 = widgets.Dropdown(options=['Decision 7'],
                value='Decision 7', description='Decision 9', disabled=False)

    drop8 = widgets.Dropdown(options=['Decision 8'],
                value='Decision 8', description='Decision 10', disabled=False)

    drop9 = widgets.Dropdown(options=['Decision 9'],
                value='Decision 9', description='Decision 11', disabled=False)
    
    drop10 = widgets.Dropdown(options=['Segment Type', 'Stable','Transitional','Break'],
                value='Segment Type', description='Decision 1', disabled=False)
    
    drop11 = widgets.Dropdown(options=['Change Agent'],
                value='Change Agent', description='Decision 2', disabled=False)
    
    # Display current validity of sample
    valid = widgets.Valid(value=False, description='Not Saved', readout='')

    
    # Create button for checking validity
    validate = widgets.Button(value=False, description='Validate', disabled=False,
                button_style='', icon='check')
    
    # Save the current sample, only possible when validity has been approved
    save_button = widgets.Button(value=False, description='Save', disabled=False,
                button_style='')
    
    # Load everything
    load_button = widgets.Button(value=False, description='Load', disabled=False,
                button_style='')
    
    valid_load = widgets.Valid(value=False, description='Not Loaded', readout='')
    
    # Interpreters confidence
    confidence = widgets.IntSlider(value=0, min=0, max=3, step=1, description='Confidence:',
                    disabled=False, continuous_update=False, orientation='horizontal',
                    readout=True, readout_format='d')
    
    # Notes
    notes = widgets.Textarea(value='Enter any useful or interesting information about the sample.',
                    placeholder='Enter any useful or interesting information about the sample',
                    description='Notes:', layout=widgets.Layout(width='70%'), disabled=False)
    
    # Spreadsheet information
    spreadsheet = widgets.Text(value='Google Spreadsheet Credential JSON',
                    placeholder='Google Spreadsheet Credential JSON',
                    description='Credentials:',disabled=False,
                    continuous_update=True)
    
    spreadName = widgets.Text(value='Google Spreadsheet Name',
                    placeholder='Google Spreadsheet Name', description='Name:',
                    disabled=False, continuous_update=True)
    
    # Sample path
    sampleWidget = widgets.Text(value='Path to sample feature collection',
                description='Path:',
                disabled=False, continuous_update=True)
    
    def __init__(self):
        interface.sample_path = None
        interface.sheet = None
  
    
    def drop1_clicked(selection):
        """ Generate secondary class selector after initial class is chosen """
        if selection.new == 'No':
            interface.drop2.set_trait('options', ['>30% Vegetated?','Yes', 'No'])
        elif selection.new == 'Yes':
            interface.drop2.set_trait('options', ['Ice/Snow'])

    def drop2_clicked(selection):
        """ Generate secondary class selector after initial class is chosen """
        if '>30% Vegetated?' in interface.drop2.options:
            if selection.new == 'Yes':
                interface.drop3.set_trait('options', ['Density','Closed (60-70%)', 'Open (30-60%)', 'Sparse (<30%)'])
                interface.drop4.set_trait('options', ['Veg Type 1','Cropland', 'Plantation', 'Wetland','Riparian/Floodplain','Other'])
                interface.drop5.set_trait('options', ['Trees?','Yes', 'No'])

            elif selection.new == 'No':
                interface.drop3.set_trait('options', ['Dominant Cover?', 'Water','Bare','Developed'])
                interface.drop4.set_trait('options', ['Decision 4'])
                interface.drop5.set_trait('options', ['Decision 5'])
                interface.drop6.set_trait('options', ['Decision 6'])
                interface.drop7.set_trait('options', ['Decision 7'])
                interface.drop8.set_trait('options', ['Decision 8'])

        else:
            interface.drop3.set_trait('options', ['No Other Information Needed'])

    def drop3_clicked(selection):
        """ Generate third class selector after initial class is chosen """
        if 'Dominant Cover?' in interface.drop3.options:
            if selection.new == 'Water':
                interface.drop4.set_trait('options', ['Water Type','Shore/Inter tidal', 'Shallows', 'River','Lake/Reservoir','Ocean'])
            elif selection.new == 'Bare':
                interface.drop4.set_trait('options', ['Bare Type', 'Soil','Rock','Quarry (Active)','Beach/Sand'])
            elif selection.new == 'Developed':
                interface.drop4.set_trait('options', ['Surface Albedo', 'High','Low','Mixed'])
                interface.drop5.set_trait('options', ['Use','Residential', 'Commercial/Industrial'])
                interface.drop6.set_trait('options', ['Building Height','No Buildings','1-2 Stories', '3-5 Stories','5+ Stories'])
                interface.drop7.set_trait('options', ['Transport','Road','Not Applicable'])
                interface.drop8.set_trait('options', ['% Impervious','High (60-100)','Medium (30-60)','Low (<30)'])

    def drop4_clicked(selection):
        """ Generate fourth class selector after initial class is chosen """
        if selection.new == 'Surface Albedo':
            interface.drop5.set_trait('options', ['Use','Residential', 'Commercial/Industrial'])
        elif selection.new == 'Bare':
            interface.drop5.set_trait('options', ['Bare Type', 'Soil','Rock','Quarry (Active)','Beach/Sand'])
        elif selection.new == 'Developed':
            interface.drop5.set_trait('options', ['Surface Albedo', 'High','Low','Mixed'])

    def drop5_clicked(selection):
        """ Generate Fifth class selector after initial class is chosen """
        if 'Trees?' in interface.drop5.options:
            if selection.new == 'Yes':
                interface.drop6.set_trait('options', ['Height >5m & Canopy >30%','Yes', 'No'])
            elif selection.new == 'No':
                interface.drop6.set_trait('options', ['Herbaceous Type','Grassland', 'Pasture','Lawn/Urban Grass','Moss/Lichen'])
   
    def drop6_clicked(selection):
        """ Generate sixth class selector after initial class is chosen """
        if 'Height >5m & Canopy >30%' in interface.drop6.options:
            if selection.new == 'Yes':
                interface.drop7.set_trait('options', ['Forest Type','Evergreen', 'Deciduous','Mixed'])
                interface.drop8.set_trait('options', ['Leaf Type','Broad', 'Needle','Unsure'])
                interface.drop9.set_trait('options', ['Location','Interior', 'Edge'])

            elif selection.new == 'No':
                interface.drop7.set_trait('options', ['Shrub Type','Evergreen', 'Deciduous','Mixed'])

    def drop10_clicked(selection):
        """ Generate change agent selector after initial class is chosen """
        if selection.new == 'Stable':
            interface.drop11.set_trait('options', ['No Change Information Needed'])
        elif selection.new == 'Transitional':
            interface.drop11.set_trait('options', ['Transitional Type','Deforestation/Logging', 'Fire','Insect Damage','Urban Development',
                                                  'Decline/Degradation','Regrowth','Other','Unsure'])
        elif selection.new == 'Break':
            interface.drop11.set_trait('options', ['Transitional Type','Deforestation/Logging', 'Fire','Insect Damage','Urban Development',
                                                  'Decline/Degradation','Regrowth','Other','Unsure'])
                

    def check_val_status(selection):
        """ Check the validity of the current sample and change valid widget accordingly """
        selected_secondary_lc = False
        wrote_correct_lc = False
        if interface.second_class_drop.value != 'Secondary Class Information':
            selected_secondary_lc = True
        else:
            print("Must specify secondary class information!")
        if interface.lc.value.capitalize() == interface.textClass.value.capitalize():
            wrote_correct_lc = True
        if selected_secondary_lc and wrote_correct_lc:
            interface.valid.value = True
            interface.save_button.disabled = False
            #interface.valid.set_trait('value',True)
            #interface.save_button.set_trait('disabled',False)
  
    
    def save_sample(selection):
        """ Save the sample to the database """
        # TODO: This should be backed up somehow
        
        # Connect to the database
        conn = sqlite3.connect(interface.dbPath)
        c = conn.cursor()
        
        # Get everything in right format
        year1 = interface.years.value[0]
        year2 = interface.years.value[1]
        
        waterType = 'N/A'
        bareType = 'N/A'
        albedo = 'N/A'
        use = 'N/A'
        height = 'N/A'
        transport = 'N/A'
        impervious = 'N/A'
        density = 'N/A'
        vegType1 = 'N/A'
        herbaceousType = 'N/A'
        shrubType = 'N/A'
        forestPhenology = 'N/A'
        leafType = 'N/A'
        location = 'N/A'
        
        condition = interface.drop10.value
        changeAgent = interface.drop11.value
        
        # Ice/Snow
        if interface.drop1.value == 'Yes':
            class1 = 'Snow/Ice'
        else:
            if interface.drop2.value == 'No': #Non-Veg
                class1 = interface.drop3.value
                if class1 == 'Water':
                    waterType = interface.drop4.value
                elif class1 == 'Bare':
                    bareType = interface.drop4.value
                else:
                    albedo = interface.drop4.value
                    use = interface.drop5.value
                    height = interface.drop6.value
                    transport = interface.drop7.value
                    impervious = interface.drop8.value
            elif interface.drop2.value == 'Yes': #Veg
                density = interface.drop3.value
                vegType1 = interface.drop4.value #herbac
                if interface.drop5.value == 'No': #Herbaceous
                    class1 = 'Herbaceous'
                    herbaceousType = interface.drop6.value
                elif interface.drop5.value == 'Yes':
                    if interface.drop6.value == 'No':
                        class1 = 'Shrub'
                        shrubType = interface.drop7.value
                    elif interface.drop6.value == 'Yes':
                        class1 = 'Forest'
                        forestPhenology = interface.drop7.value
                        leafType = interface.drop8.value
                        location = interface.drop9.value
                
        conf = interface.confidence.value
        notes_value = interface.notes.value
        idSample = Plot_interface.current_id
        lat = Plot_interface.m.center[0]
        lon = Plot_interface.m.center[1]
        

        sampleInput = (idSample, lat, lon, year1, year2, condition, changeAgent, class1, waterType, bareType, albedo, use, height, transport, impervious, density,
                       vegType1, herbaceousType, shrubType, forestPhenology, leafType, location, conf, notes_value)

        # Put sample information into database
        c.execute("""insert into sample3
                  values {i}""".format(i=sampleInput))

        # Save (commit) the changes
        conn.commit()

        # Close the cursor
        c.close()
        
        # Change save validity state
        interface.valid.value = True
        interface.valid.description='Saved!'

        # Reset the buttons
        interface.years.set_trait('value',[2012, 2015])
        
        # Save to drive
        sampleInputList = [idSample, lat, lon, year1, year2, condition, changeAgent, class1, waterType, bareType, albedo, use, height, transport, impervious, density,
                           vegType1, herbaceousType, shrubType, forestPhenology, leafType, location, conf, notes_value]
        interface.sheet.insert_row(sampleInputList, 2)

    def load_everything(sender):
        # Load Google Spreadsheet and GEE Feature Collection
        scope = ['https://spreadsheets.google.com/feeds' + ' ' +'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name(interface.spreadsheet.value, scope)
        client = gspread.authorize(creds)
        interface.sheet = client.open(interface.spreadName.value).sheet1
        interface.sample_path = ee.FeatureCollection(interface.sampleWidget.value)
        interface.fc_df = fc2dfgeo(interface.sample_path)
        if 'ID' not in interface.fc_df.columns:
            first_index = interface.fc_df.index.min()
            interface.fc_df['ID'] = interface.fc_df.index
        else:
            first_index = interface.fc_df['ID'].min()
        interface.fc_df = interface.fc_df.set_index('ID')
        interface.valid_load.value = True
        interface.valid_load.description='Loaded!'
        Plot_interface.current_id = first_index - 1
        
    # Handle widget interactions
    # Call function check_val_status when button is clicked
    validate.on_click(check_val_status)    
    # Primary class selector action
    drop1.observe(drop1_clicked, 'value')
    # Secondary class selector action
    drop2.observe(drop2_clicked, 'value')
    # Third class selector action
    drop3.observe(drop3_clicked, 'value')
    drop4.observe(drop4_clicked, 'value')
    drop5.observe(drop5_clicked, 'value')
    drop6.observe(drop6_clicked, 'value')
    drop10.observe(drop10_clicked, 'value')

    # Call function save_sample when clicking save button
    save_button.on_click(save_sample)
    
    # Load database and sample
    load_button.on_click(load_everything)


In [104]:
class Plot_interface(object):
    """Class to handle map and plot interaction"""

    # Declare class attributes
    pyccd_flag = False
    pyccd_flag2 = False
    current_band = ''
    band_index1 = 4
    band_index2 = 4
    click_col = ''
    point_color = ['#43a2ca']
    click_df = pd.DataFrame()
    sample_col = ''
    sample_df = pd.DataFrame()
    PyCCDdf = pd.DataFrame()
    table = pd.DataFrame()
    band_list =['BLUE', 'GREEN', 'RED','NIR','SWIR1','SWIR2']
    year_range = [ 1986, 2018 ]
    doy_range = [ 1, 365 ]
    step = 1 #in years
    
    # Create widget controls
    next_pt = Button(value=False, description='Next point', disabled=False) 
    previous_pt = Button(value=False, description='Previous point', disabled=False)
    pyccd_button = Button(value=False, description='Run PyCCD 1', disabled=False)
    pyccd_button2 = Button(value=False, description='Run PyCCD 2', disabled=False)
    band_selector1 = Dropdown(options=['BLUE', 'GREEN', 'RED','NIR','SWIR1','SWIR2'], 
                             description='Select band', value=None)
    band_selector2 = Dropdown(options=['BLUE', 'GREEN', 'RED','NIR','SWIR1','SWIR2'], 
                             description='Select band', value=None)
    image_band_1 = Dropdown(options=['BLUE', 'GREEN', 'RED','NIR','SWIR1','SWIR2'], 
                             description='Red:', value='SWIR1')
    image_band_2 = Dropdown(options=['BLUE', 'GREEN', 'RED','NIR','SWIR1','SWIR2'], 
                         description='Green:', value='NIR')
    image_band_3 = Dropdown(options=['BLUE', 'GREEN', 'RED','NIR','SWIR1','SWIR2'], 
                             description='Blue:', value='RED')
    stretch_min = FloatText(value=0, description='Min:', disabled=False)
    stretch_max = FloatText(value=6000, description='Min:', disabled=False)
    
    # Clear layers on map
    clear_layers = Button(value=False, description='Clear Map', disabled=False) 

    # Color points by DOY
    color_check = widgets.Checkbox(
                    value=False,
                    description='Color DOY',
                    disabled=False
    )
    
    idBox = widgets.Text(
                value='0',
                description='ID:',
                disabled=False
            )
    
    ylim = widgets.IntRangeSlider(value=[0, 4000], min=0, max=10000,
            step=500, description='YLim:', disabled=False, continuous_update=False,
            orientation='horizontal', readout=True, readout_format='d')
    
    xlim = widgets.IntRangeSlider(value=[2000, 2018], min=1986, max=2018,
            step=1, description='XLim:', disabled=False, continuous_update=False,
            orientation='horizontal', readout=True, readout_format='d')
    
    ylim2 = widgets.IntRangeSlider(value=[0, 4000], min=0, max=10000,
            step=500, description='YLim:', disabled=False, continuous_update=False,
            orientation='horizontal', readout=True, readout_format='d')
    
    xlim2 = widgets.IntRangeSlider(value=[2000, 2018], min=1986, max=2018,
            step=1, description='XLim:', disabled=False, continuous_update=False,
            orientation='horizontal', readout=True, readout_format='d')
    
    
    coords_label = Label()
    pt_message = HTML("Current ID: ")
    time_label = HTML(value='')
    selected_label = HTML("ID of selected point")
    hover_label = HTML("test value")
    text_brush = HTML(value = 'Selected year range:')
    kml_link = HTML(value = 'KML:')
    error_label = HTML(value = 'Load a point')

    
    # Create map including streets and satellite and controls
    m = ipyleaflet.Map(zoom=5, layout={'height':'400px'}, 
                       center=(3.3890701010382958, -67.32297252983098),dragging=True,
                       close_popup_on_click=False, basemap=ipyleaflet.basemaps.Esri.WorldStreetMap)
    
    streets = ipyleaflet.basemap_to_tiles(ipyleaflet.basemaps.Esri.WorldImagery)
    m.add_layer(streets)

    dc = ipyleaflet.DrawControl(marker={'shapeOptions': {'color': '#ff0000'}}, 
                                polygon={}, circle={}, circlemarker={}, polyline={})
    
    
    # Table widget
    table_widget = qgrid.show_grid(table, show_toolbar=False)
    
    # Set plots
    # Plot scales.
    lc1_x = bqplot.DateScale(min=datetime.date(xlim.value[0], 2, 1), max=datetime.date(xlim.value[1], 1, 1))
    lc2_y = bqplot.LinearScale(min=ylim.value[0], max=ylim.value[1])
    
    lc1_x2 = bqplot.DateScale(min=datetime.date(xlim.value[0], 2, 1), max=datetime.date(xlim.value[1], 1, 1))
    lc2_y2 = bqplot.LinearScale(min=ylim.value[0], max=ylim.value[1])

    # Main scatter plot for samples
    lc2 = bqplot.Scatter(
        x=[],
        y=[],
        scales={'x': lc1_x, 'y': lc2_y},
        size=[1,1],
        interactions={'click': 'select', 'hover': 'tooltip'},
        selected_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
        unselected_style={'opacity': 0.5},
        display_legend=True,
        labels = ['Sample point']
    )
    
    # Pyccd model fit
    lc4 = bqplot.Lines(
        x=[],
        y=[],
        colors=['black'],
        stroke_width=3,
        scales={'x': lc1_x, 'y': lc2_y},
        size=[1,1],
    )
    
    # Pyccd model break
    lc5 = bqplot.Scatter(
        x=[],
        y=[],
        marker='triangle-up',
        colors=['red'],
        scales={'x': lc1_x, 'y': lc2_y},
        size=[1,1],
        display_legend=False,
        labels = ['Model Endpoint']
    )
    
    # Scatter plot for clicked points in map
    lc3 = bqplot.Scatter(
        x=[],
        y=[],
        scales={'x': lc1_x2, 'y': lc2_y2},
        size=[1,1],
        colors=['gray'],
        interactions={'click': 'select', 'hover': 'tooltip'},
        selected_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
        unselected_style={'opacity': 0.5},
        display_legend=True,
        labels = ['Clicked point']
    )
    
    # Pyccd model fit for clicked point
    lc6 = bqplot.Lines(
        x=[],
        y=[],
        colors=['black'],
        stroke_width=3,
        scales={'x': lc1_x2, 'y': lc2_y2},
        size=[1,1],
    )
    
    # Pyccd model break for clicked point
    lc7 = bqplot.Scatter(
        x=[],
        y=[],
        marker='triangle-up',
        colors=['red'],
        scales={'x': lc1_x2, 'y': lc2_y2},
        size=[1,1],
        display_legend=False,
        labels = ['Model Endpoint']
    )

    # Plot axes.
    x_ax1 = bqplot.Axis(label='Date', scale=lc1_x, num_ticks = 6, tick_format='%Y')
    x_ax2 = bqplot.Axis(label='Date', scale=lc1_x2, num_ticks = 6, tick_format='%Y')
    x_ay1 = bqplot.Axis(label='SWIR1', scale=lc2_y, orientation='vertical')
    x_ay2 = bqplot.Axis(label='SWIR1', scale=lc2_y2, orientation='vertical')


    # Create a figure for sample points.
    fig = bqplot.Figure(
        marks=[lc2, lc4, lc5],
        axes=[x_ax1, x_ay1],
        layout={'height':'300px', 'width':'100%'},
        title="Sample TS"
    )
    
    # Create a figure for clicked points.
    fig2 = bqplot.Figure(
        marks=[lc3, lc6, lc7],
        axes=[x_ax2, x_ay2],
        layout={'height':'300px', 'width':'100%'},
        title="Clicked TS"
    )

    def __init__(self, navigate):
        Plot_interface.navigate = navigate
        Plot_interface.band_index1 = 4
        Plot_interface.band_index2 = 4
        Plot_interface.pyccd_flag = False
        Plot_interface.pyccd_flag2 = False
        Plot_interface.table = None  
        # Set up database
        conn = sqlite3.connect(Plot_interface.navigate.dbPath)
        Plot_interface.c = conn.cursor()
        Plot_interface.minv = 0
        Plot_interface.maxv = 6000
        Plot_interface.b1 = 'SWIR1'
        Plot_interface.b2 = 'NIR'
        Plot_interface.b3 = 'RED'
           
    @classmethod
    def map_point(self):
        gjson = ipyleaflet.GeoJSON(data=Plot_interface.navigate.fc_df['geometry'][Plot_interface.current_id], name="Sample point")
        Plot_interface.m.center = gjson.data['coordinates'][::-1]
        Plot_interface.m.zoom = 12
        Plot_interface.m.add_layer(gjson)
        kmlstr = ee.FeatureCollection(ee.Geometry.Point(Plot_interface.navigate.fc_df['geometry'][Plot_interface.current_id]['coordinates'])).getDownloadURL("kml")
        Plot_interface.kml_link.value = "<a '_blank' rel='noopener noreferrer' href={}>KML Link</a>".format(kmlstr)
        
    @classmethod
    def get_ts(self):
        try:
            Plot_interface.error_label.value = 'Loading'
            Plot_interface.current_band = Plot_interface.band_list[Plot_interface.band_index1]
            Plot_interface.sample_col = get_full_collection(Plot_interface.navigate.fc_df['geometry'][Plot_interface.current_id]['coordinates'], 
                                                              Plot_interface.year_range, Plot_interface.doy_range)
            Plot_interface.sample_df = get_df_full(Plot_interface.sample_col, Plot_interface.navigate.fc_df['geometry'][Plot_interface.current_id]['coordinates']).dropna()
            Plot_interface.error_label.value = 'Point loaded!'
        except:
            Plot_interface.error_label.value = 'Point could not be loaded!'
 
    def clear_map(b):
        Plot_interface.m.clear_layers()
        Plot_interface.m.add_layer(Plot_interface.streets)
        Plot_interface.map_point()

    
    @classmethod
    def plot_ts(self):
        current_band = Plot_interface.band_list[Plot_interface.band_index1]
        Plot_interface.lc2.x = Plot_interface.sample_df['datetime']
        if Plot_interface.color_check.value == False:
            Plot_interface.lc2.colors = list(Plot_interface.point_color)
        else:
            Plot_interface.lc2.colors = list(Plot_interface.sample_df['color'].values)
        Plot_interface.lc2.y = Plot_interface.sample_df[current_band]
        Plot_interface.x_ay1.label = current_band
        Plot_interface.lc4.x = []
        Plot_interface.lc4.y = []
        Plot_interface.lc5.x = []
        Plot_interface.lc5.y = []
        #if pyccd_flag:
        if Plot_interface.pyccd_flag:
            Plot_interface.run_pyccd(0)
        
    # Go back or forth between sample points
    def advance(b):
        # Plot point in map
        Plot_interface.lc4.x = []
        Plot_interface.lc4.y = []
        Plot_interface.lc5.x = []
        Plot_interface.lc5.y = []
        Plot_interface.lc5.display_legend=False
        Plot_interface.pyccd_flag = False
        Plot_interface.current_id += 1
        Plot_interface.pt_message.value = "Point ID: {}".format(Plot_interface.current_id)
        Plot_interface.map_point() 
        Plot_interface.get_ts()
        Plot_interface.plot_ts()
        Plot_interface.change_table()
        Plot_interface.navigate.valid.value = False
        Plot_interface.navigate.description='Not Saved'
        
    def decrease(b):
        # Plot point in map
        Plot_interface.lc4.x = []
        Plot_interface.lc4.y = []
        Plot_interface.lc5.x = []
        Plot_interface.lc5.y = []
        Plot_interface.lc5.display_legend=False
        Plot_interface.pyccd_flag = False
        Plot_interface.current_id -= 1
        Plot_interface.pt_message.value = "Point ID: {}".format(Plot_interface.current_id)
        Plot_interface.map_point() 
        Plot_interface.get_ts()
        Plot_interface.plot_ts()    
        Plot_interface.change_table()
        Plot_interface.navigate.valid.value = False
        Plot_interface.navigate.description='Not Saved'
        
    def change_table():
        # Update the table based on current ID

        # Get header
        cursor = Plot_interface.c.execute('select * from sample3')
        names = list(map(lambda x: x[0], cursor.description))
        previous_inputs = pd.DataFrame()
        for i, row in enumerate(Plot_interface.c.execute("SELECT * FROM sample3 WHERE id = '%s'" % Plot_interface.current_id)):
            previous_inputs[i] = row
        previous_inputs = previous_inputs.T
        if previous_inputs.shape[0] > 0:
            previous_inputs.columns = names
        Plot_interface.table_widget.df = previous_inputs

    # Functions for changing image stretch    
    def change_image_band1(change):        
        new_band = change['new']
        Plot_interface.b1 = new_band
    def change_image_band2(change):        
        new_band = change['new']
        Plot_interface.b2 = new_band
    def change_image_band3(change):        
        new_band = change['new']
        Plot_interface.b3 = new_band
    
    # Band selection for sample point
    def on_band_selection1(change):        
        new_band = change['new']
        #global band_index
        band_index = change['owner'].index
        Plot_interface.band_index1 = band_index
        Plot_interface.plot_ts()

    # Band selection for clicked point
    def on_band_selection2(change):        
        new_band = change['new']
        band_index = change['owner'].index
        Plot_interface.band_index2 = band_index
        Plot_interface.lc3.x = Plot_interface.click_df['datetime']
        Plot_interface.lc3.y = Plot_interface.click_df[new_band]
        Plot_interface.x_ay2.label = new_band
        if Plot_interface.pyccd_flag2:
            Plot_interface.run_pyccd2(0)

    def change_yaxis(value):
        Plot_interface.lc2_y.min = Plot_interface.ylim.value[0]
        Plot_interface.lc2_y.max = Plot_interface.ylim.value[1]

    def change_xaxis(value):
        Plot_interface.lc1_x.min = datetime.date(Plot_interface.xlim.value[0], 2, 1)
        Plot_interface.lc1_x.max = datetime.date(Plot_interface.xlim.value[1], 2, 1)
        Plot_interface.year_range = [Plot_interface.xlim.value[0], Plot_interface.xlim.value[1]]
    
    def change_yaxis2(value):
        Plot_interface.lc2_y2.min = Plot_interface.ylim2.value[0]
        Plot_interface.lc2_y2.max = Plot_interface.ylim2.value[1]

    def change_xaxis2(value):
        Plot_interface.lc1_x2.min = datetime.date(Plot_interface.xlim2.value[0], 2, 1)
        Plot_interface.lc1_x2.max = datetime.date(Plot_interface.xlim2.value[1], 2, 1)
        
    
    def hover_event(self, target):
        Plot_interface.hover_label.value = str(target['data']['x'])
    
    # Add layer from clicked point in sample TS figure
    def click_event(self, target):
        pt_index = target['data']['index']
        current_band = Plot_interface.band_list[Plot_interface.band_index1]
        image_id = Plot_interface.sample_df['id'].values[pt_index]
        selected_image = ee.Image(Plot_interface.sample_col.filterMetadata('system:index', 'equals', image_id).first())
        tile_url = GetTileLayerUrl(selected_image.visualize(min=Plot_interface.stretch_min.value, 
                                                            max=Plot_interface.stretch_max.value, 
                                                            bands= [Plot_interface.b1, Plot_interface.b2, 
                                                                    Plot_interface.b3]))

        Plot_interface.m.add_layer(ipyleaflet.TileLayer(url=tile_url, name=image_id))

    # Add layer from clicked point in clicked TS figure
    def click_event2(self, target):
        pt_index = target['data']['index']
        current_band = Plot_interface.band_list[Plot_interface.band_index2]
        #Find clicked image. .values needed to access the nth element of that list instead of indexing by ID
        image_id = Plot_interface.click_df['id'].values[pt_index]
        selected_image = ee.Image(Plot_interface.click_col.filterMetadata('system:index', 'equals', image_id).first())
        tile_url = GetTileLayerUrl(selected_image.visualize(min=Plot_interface.minv, 
                                                            max=Plot_interface.maxv, 
                                                            bands= [Plot_interface.b1, Plot_interface.b2, 
                                                                    Plot_interface.b3]))

        Plot_interface.m.add_layer(ipyleaflet.TileLayer(url=tile_url, name=image_id))
    
    # Plot TS from clicked point ERIC HERE
    def handle_draw(self, action, geo_json):
        # Get the selected coordinates from the map's drawing control.
        current_band = Plot_interface.band_list[Plot_interface.band_index2]
        coords = geo_json['geometry']['coordinates']    
        Plot_interface.click_col = get_full_collection(coords, Plot_interface.year_range, Plot_interface.doy_range)
        Plot_interface.click_df = get_df_full(Plot_interface.click_col, coords).dropna()
        Plot_interface.lc6.x = []
        Plot_interface.lc6.y = []
        Plot_interface.lc7.x = []
        Plot_interface.lc7.y = []
        Plot_interface.lc3.x = Plot_interface.click_df['datetime']
        Plot_interface.lc3.y = Plot_interface.click_df[current_band]
        
        if Plot_interface.color_check.value == False:
            Plot_interface.lc3.colors = list(Plot_interface.point_color)
        else:
            Plot_interface.lc3.colors = list(Plot_interface.click_df['color'].values)
    
    # Plotting pyccd
    def plot_pyccd(results, band, plotband, dates, yl, ylabel, ts_type):
        mask = np.array(results['processing_mask']).astype(np.bool_)
        predicted_values = []
        prediction_dates = []
        break_dates = []
        start_dates = []

        for num, result in enumerate(results['change_models']):
            days = np.arange(result['start_day'], result['end_day'] + 1)
            prediction_dates.append(days)
            break_dates.append(result['break_day'])
            start_dates.append(result['start_day'])
            intercept = result[list(result.keys())[6+band]]['intercept']
            coef = result[list(result.keys())[6+band]]['coefficients']

            predicted_values.append(intercept + coef[0] * days +
                                    coef[1]*np.cos(days*1*2*np.pi/365.25) + coef[2]*np.sin(days*1*2*np.pi/365.25) +
                                    coef[3]*np.cos(days*2*2*np.pi/365.25) + coef[4]*np.sin(days*2*2*np.pi/365.25) +
                                    coef[5]*np.cos(days*3*2*np.pi/365.25) + coef[6]*np.sin(days*3*2*np.pi/365.25))

        num_breaks = len(break_dates)

        break_y = [plotband[dates == i][0] for i in break_dates]

        #break_y = [0] * num_breaks
        break_dates_plot = [datetime.datetime.fromordinal(i).strftime('%Y-%m-%d %H:%M:%S.%f') for i in break_dates]

        plot_dates = np.array([datetime.datetime.fromordinal(i) for i in (dates)])

        # Predicted curves
        all_dates = []
        all_preds = []
        for _preddate, _predvalue in zip(prediction_dates, predicted_values):
            all_dates.append(_preddate)
            all_preds.append(_predvalue)

        all_preds = [item for sublist in all_preds for item in sublist]
        all_dates = [item for sublist in all_dates for item in sublist]

        date_ord = [datetime.datetime.fromordinal(i).strftime('%Y-%m-%d %H:%M:%S.%f') for i in all_dates]
        _x = np.array(date_ord, dtype='datetime64')
        _y = all_preds
        
        if ts_type == 'sample_ts':
            Plot_interface.lc4.x = _x
            Plot_interface.lc4.y = _y
            Plot_interface.lc5.x = np.array(break_dates_plot, dtype='datetime64')
            Plot_interface.lc5.y = break_y
        elif ts_type == 'clicked_ts':
            Plot_interface.lc6.x = _x
            Plot_interface.lc6.y = _y
            Plot_interface.lc7.x = np.array(break_dates_plot, dtype='datetime64')
            Plot_interface.lc7.y = break_y
            
    # Go to a specific sample
    def go_to_sample(b):
        # Plot point in map
        Plot_interface.lc4.x = []
        Plot_interface.lc4.y = []
        Plot_interface.lc5.x = []
        Plot_interface.lc5.y = []
        Plot_interface.lc5.display_legend=False
        Plot_interface.pyccd_flag = False
        Plot_interface.current_id = int(b.value)
        Plot_interface.pt_message.value = "Point ID: {}".format(Plot_interface.current_id)
        Plot_interface.navigate.valid.value = False
        Plot_interface.navigate.description='Not Saved'
        Plot_interface.map_point() 
        Plot_interface.get_ts()
        Plot_interface.plot_ts()        

    # Run pyccd
    def run_pyccd(b):
        # Run pyCCD on current point
        Plot_interface.pyccd_flag = True
        Plot_interface.lc5.display_legend=True
        dfPyCCD = Plot_interface.sample_df

        dfPyCCD['pixel_qa'][dfPyCCD['pixel_qa'] > 4] = 0

        #TODO: Paramaterize everything
        params = {'QA_BITPACKED': False,
                  'QA_FILL': 255,
                  'QA_CLEAR': 0,
                  'QA_WATER': 1,
                  'QA_SHADOW': 2,
                  'QA_SNOW': 3,
                  'QA_CLOUD': 4}

        dates = np.array(dfPyCCD['ord_time'])
        blues = np.array(dfPyCCD['BLUE'])
        greens = np.array(dfPyCCD['GREEN'])
        reds = np.array(dfPyCCD['RED'])
        nirs = np.array(dfPyCCD['NIR'])
        swir1s = np.array(dfPyCCD['SWIR1'])
        swir2s = np.array(dfPyCCD['SWIR2'])
        thermals = np.array(dfPyCCD['THERMAL'])
        qas = np.array(dfPyCCD['pixel_qa'])
        results = ccd.detect(dates, blues, greens, reds, nirs, swir1s, swir2s, thermals, qas, params=params)

        band_names = ['Blue SR', 'Green SR', 'Red SR', 'NIR SR', 'SWIR1 SR', 'SWIR2 SR','THERMAL']
        plotlabel = band_names[Plot_interface.band_index1] 

        plot_arrays = [blues, greens, reds, nirs, swir1s, swir2s]
        plotband = plot_arrays[Plot_interface.band_index1]
        Plot_interface.plot_pyccd(results, Plot_interface.band_index1, 
                                  plotband, dates, (0, 4000), 'PyCCD Results', 'sample_ts')
    
    def run_pyccd2(b):
        # Run pyCCD on current point
        Plot_interface.pyccd_flag2 = True
        
        # Display the legend
        Plot_interface.lc7.display_legend=True
        
        dfPyCCD = Plot_interface.click_df

        # First two lines no longer required bc we are removing NA's when we load the TS
        dfPyCCD['pixel_qa'][dfPyCCD['pixel_qa'] > 4] = 0

        #TODO: Paramaterize everything
        params = {'QA_BITPACKED': False,
                  'QA_FILL': 255,
                  'QA_CLEAR': 0,
                  'QA_WATER': 1,
                  'QA_SHADOW': 2,
                  'QA_SNOW': 3,
                  'QA_CLOUD': 4}

        dates = np.array(dfPyCCD['ord_time'])
        blues = np.array(dfPyCCD['BLUE'])
        greens = np.array(dfPyCCD['GREEN'])
        reds = np.array(dfPyCCD['RED'])
        nirs = np.array(dfPyCCD['NIR'])
        swir1s = np.array(dfPyCCD['SWIR1'])
        swir2s = np.array(dfPyCCD['SWIR2'])
        thermals = np.array(dfPyCCD['THERMAL'])
        qas = np.array(dfPyCCD['pixel_qa'])
        results = ccd.detect(dates, blues, greens, reds, nirs, swir1s, swir2s, thermals, qas, params=params)

        band_names = ['Blue SR', 'Green SR', 'Red SR', 'NIR SR', 'SWIR1 SR', 'SWIR2 SR','THERMAL']
        plotlabel = band_names[Plot_interface.band_index2] 

        plot_arrays = [blues, greens, reds, nirs, swir1s, swir2s]
        plotband = plot_arrays[Plot_interface.band_index2]
        Plot_interface.plot_pyccd(results, Plot_interface.band_index2, 
                                  plotband, dates, (0, 4000), 'PyCCD Results', 'clicked_ts')
    
    ylim.observe(change_yaxis)
    xlim.observe(change_xaxis)
    ylim2.observe(change_yaxis2)
    xlim2.observe(change_xaxis2)
    next_pt.on_click(advance)
    previous_pt.on_click(decrease)
    pyccd_button.on_click(run_pyccd)
    pyccd_button2.on_click(run_pyccd2)
    clear_layers.on_click(clear_map)
    band_selector1.observe(on_band_selection1, names='value')
    band_selector2.observe(on_band_selection2, names='value')
    
    image_band_1.observe(change_image_band1, names='value')
    image_band_2.observe(change_image_band2, names='value')
    image_band_3.observe(change_image_band3, names='value')
    
    lc2.on_element_click(click_event)
    lc2.tooltip=hover_label
    lc2.on_hover(hover_event)

    lc3.on_element_click(click_event2)
    lc3.tooltip=hover_label
    lc3.on_hover(hover_event)

    idBox.on_submit(go_to_sample)

    
    dc.on_draw(handle_draw)
    m.add_control(dc)
    m.add_control(ipyleaflet.LayersControl())

In [105]:
display(HTML('<style>.container { width:100% !important; }</style>'))

HTML(value='<style>.container { width:100% !important; }</style>')

In [106]:
newInterface = interface()
newPlot = Plot_interface(newInterface)

navigate_samples = HBox(children=[newPlot.previous_pt, newPlot.next_pt])
load_data = HBox(children=[newInterface.load_button, newInterface.valid_load])
tab0 = VBox(children=[newInterface.spreadsheet, newInterface.spreadName, newInterface.sampleWidget,
                      load_data])
dropBox1= VBox(children=[newInterface.drop10,
                         newInterface.drop11,
                         newInterface.drop1,
                         newInterface.drop2,
                         newInterface.drop3,
                         newInterface.drop4])

dropBox2= VBox(children=[newInterface.drop5,
                         newInterface.drop6,
                         newInterface.drop7,
                         newInterface.drop8,
                         newInterface.drop9,
                         newInterface.years])
dropBoxBoth = HBox(children=[dropBox1, dropBox2])
tab1 = VBox(children=[dropBoxBoth,newInterface.notes])
sample_pyccd = HBox([newPlot.pyccd_button, newPlot.pt_message, newPlot.kml_link])
sample_ts_box = VBox([HTML(value="<b>SAMPLE TS</b>"), newPlot.band_selector1, newPlot.ylim, newPlot.xlim, navigate_samples, sample_pyccd])
clicked_ts_box = VBox([HTML(value="<b>CLICKED TS</b>"), newPlot.band_selector2, newPlot.ylim2, newPlot.xlim2, newPlot.pyccd_button2, newPlot.idBox])
ts_tab = HBox([sample_ts_box, clicked_ts_box])
stretch_tab = VBox([Plot_interface.image_band_1, Plot_interface.image_band_2, 
                    Plot_interface.image_band_3, Plot_interface.stretch_min,
                    Plot_interface.stretch_max, Plot_interface.color_check,
                    Plot_interface.clear_layers])
tab2 = widgets.Tab(children=[ts_tab, stretch_tab], layout=Layout(width='50%', min_height='270px', max_height='270px'))

tab2.set_title(0, 'Controls')
tab2.set_title(1, 'Display')
tab3 = VBox(children=[newInterface.confidence,
                      newInterface.save_button,
                      newInterface.valid,
                      newPlot.error_label])
tab4 = VBox(children=[newPlot.table_widget])

tab = widgets.Tab(children=[tab0, tab1, tab3, tab4], layout=Layout(width='100%', min_height='270px', max_height='270px'))
tab.set_title(0, 'Load sample')
tab.set_title(1, 'Classification')
tab.set_title(2, 'Interpretation')
tab.set_title(3, 'Table')

display(newPlot.m)
fig = HBox(children=[newPlot.fig, newPlot.fig2])
controls = HBox([tab2, tab])
tabfig = VBox([controls, fig])
display(tabfig)


Map(basemap={'url': 'http://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{…